### Q31. requestsライブラリを利用し、Yahoo!ファイナンス業種別銘柄一覧：サービス業のトップページのhtmlを取得し表示せよ。

In [1]:
import requests

url = "https://finance.yahoo.co.jp/search/qi/?ids=9050"

request = requests.get(url)
# print(request.text)

### Q32. 課題Q31で取得したHTMLを解析して、サービス業を営む上場企業の銘柄コード、およびそれに対応する（ページアクセス時点での）株価のリストを20件を取得せよ。

In [2]:
from pyquery import PyQuery as pq

html = pq(request.text)
data = html('#sr').children('div').children('ul').children('li').items()

for i, element in enumerate(data):
    code = element('a')('li').text().split(' ')[0]
    price = element('a')('div span:first span').children('span').text()
    print(f'{code}: {price: >6}')

1717:    703
1954:  2,934
2120:    234
2121:  2,180
2122:    912
2124:  1,826
2127:  1,653
2130:  3,230
2134:     28
2136:    702
2137:    795
2139:    377
2146:  2,900
2148:  1,688
2150:    930
2152:  1,093
2153:  1,162
2154:  1,641
2156:    256
2157:    667


### Q33. Yahoo!ファイナンス業種別銘柄一覧：サービス業のサイトでは、1ページあたり最大20件の銘柄とその株価が表示される設計になっている。また「次の20件」をクリックすることで更に別の20件の銘柄を確認することができる。当該サイトのトップページのHTMLを解析し、ページ内に表示された「次の20件」リンクのURLを取得せよ。

### Q34. Yahoo!ファイナンス業種別銘柄一覧：サービス業のサイトには、サービス業を営む400以上の企業（以下サービス系企業）の株価情報が掲載されている。上記サイトで閲覧可能なすべてのサービス系企業の（ページアクセス時点での）株価を取得せよ。ただし、出力は下記のように、株価が高い順に、銘柄コード名、株価のペアをタプル形式で表示するものとする。

Q33,Q34については、次のページへの遷移がスクリプトで処理されるため、リンクは取得できなくなっている。Seleniumなどブラウザを直接操作するタイプのライブラリでないと対応できない？

### Q35. 任意の銘柄コードstock_codeが与えられたとき、Yahoo!ファイナンスのウェブサイトをリアルタイムに解析することで、銘柄コードstock_codeに対応する企業名、現在の株価、および前日終値を返す関数get_stock_infoを実装せよ。なお、入力引数はstock_codeとする。また、出力は{'company': 企業名, "current_stock_price":現在の株価, "last_close":前日終値}という辞書形式で返すこと。また、出力は存在しない銘柄コードが入力された場合はNoneオブジェクトを出力せよ。

In [84]:
def get_stock_info(stock_code):
    url = 'https://finance.yahoo.co.jp/quote/' + stock_code
    request = requests.get(url)
    html = pq(request.text)
    data01 = html('div._1nb3c4wQ').children('header')
    data02 = html('dl._38iJU1zx').items()
    data02 = pq([item for item in data02][0])

    company = data01('h1').text()
    current_stock_price = data01('span:first').text()
    last_close = data02('dd').children('span:first span span').text()
    result = {
        'company': company,
        'current_stock_price': current_stock_price,
        'last_close': last_close
    }
    return result

In [85]:
get_stock_info('4689.T')

{'company': 'Ｚホールディングス(株)',
 'current_stock_price': '504.4',
 'last_close': '504.7'}

### Q.36 CiNii Books APIを利用し、「お好み焼き」というキーワードに関する図書・雑誌リストを10件取得し、そのタイトルを出力せよ。

ヒント1：CiNii Books APIに書かれているように、 http://ci.nii.ac.jp/books/opensearch/search?(パラメータ=値)&(パラメータ=値)&…&(パラメータ=値) にHTTPアクセスすることで、検索結果を取得することができる。  
ヒント2：最低限使用するクエリパラメータは「フリーワードq」と「出力フォーマットformat」の2つ。

In [102]:
cn_endpoint = 'http://ci.nii.ac.jp/books/opensearch/search/'
query = {
    'q': 'お好み焼き',
    'format': 'json',
    'count': 10
 }

response = requests.get(cn_endpoint, params=query)
data = response.json()

for item in data['@graph'][0]['items']:
    print(item['title'])

お好み焼き食べたい : 具とソースで満腹バリエ33種
広島ベストスポット
お好み焼きの物語 : 執念の調査が解き明かす新戦前史
神戸とお好み焼き : まちづくりと比較都市論の視点から
お好み焼きたこ焼きもんじゃ焼き&鉄板焼き
「街的 (まちてき)」ということ : お好み焼き屋は街の学校だ
浜内千波の野菜たっぷりやせる!お好み焼きダイエット
お好み焼き90%たこ焼き10%の本 : 粉もん100%レシピ&店ガイド
絶品お好み焼き : たこ焼き・もんじゃ焼き : 人気店の味を家で楽しむ
お好み焼き繁盛店「鶴橋風月」成功のキーワード100


### Q37. CiNii Books APIを利用し、静岡県の図書館に蔵書されている図書・雑誌のリストを10000件取得し、出版年ごとに図書・雑誌のタイトル数を出力せよ。なお、出力は出版年が若い順に出版年、タイトル数のペアを表示するものとする。また、「19--」のように出版年が一部欠損しているものは出力結果から除外するものとする。

In [112]:
cn_endpoint = 'http://ci.nii.ac.jp/books/opensearch/search/'
query = {
    'area': 22,
    'count': 10000,
    'format': 'json'
}

response = requests.get(cn_endpoint, params=query)
data = response.json()['@graph'][0]['items']
# print(data[0]['prism:publicationDate'])

result = dict()
for item in data:
    try:
        p_date = item['prism:publicationDate']
        if(int(p_date)) in result:
            result[int(p_date)] += 1
        else:
            result[int(p_date)] = 1
    except:
        pass

result = sorted(result.items())

for item in result:
    print(item[0], ':', item[1])

1873 : 1
1875 : 1
1876 : 1
1877 : 1
1878 : 1
1881 : 1
1882 : 1
1883 : 1
1885 : 1
1886 : 1
1887 : 2
1890 : 1
1894 : 3
1895 : 5
1896 : 4
1897 : 1
1898 : 1
1899 : 2
1900 : 1
1901 : 1
1903 : 5
1904 : 2
1905 : 5
1906 : 4
1907 : 1
1908 : 8
1909 : 4
1910 : 8
1911 : 5
1912 : 3
1913 : 3
1914 : 9
1915 : 8
1916 : 5
1917 : 8
1918 : 4
1919 : 9
1920 : 6
1921 : 9
1922 : 7
1923 : 17
1924 : 32
1925 : 19
1926 : 21
1927 : 25
1928 : 25
1929 : 29
1930 : 21
1931 : 19
1932 : 28
1933 : 40
1934 : 29
1935 : 19
1936 : 31
1937 : 38
1938 : 33
1939 : 72
1940 : 98
1941 : 50
1942 : 45
1943 : 60
1944 : 60
1945 : 12
1946 : 21
1947 : 47
1948 : 81
1949 : 69
1950 : 60
1951 : 58
1952 : 69
1953 : 45
1954 : 44
1955 : 75
1956 : 84
1957 : 79
1958 : 79
1959 : 65
1960 : 57
1961 : 39
1962 : 55
1963 : 65
1964 : 96
1965 : 53
1966 : 51
1967 : 46
1968 : 39
1969 : 36
1970 : 31
1971 : 39
1972 : 38
1973 : 32
1974 : 32
1975 : 35
1976 : 32
1977 : 53
1978 : 48
1979 : 48
1980 : 53
1981 : 47
1982 : 63
1983 : 75
1984 : 68
1985 : 47
1986 : 54


### Q38. Q36の課題で実装したコードを改良し、CiNii Books APIを利用し、任意のキーワードqueryが与えられたときに、それに関する図書・雑誌リストを10件取得し、そのタイトル、著者名、および書誌詳細ページのURL（例はコチラ）を返す関数simple_search_booksを実装せよ。なお、出力は{"title":タイトル, "author":著者名, "detail_url":書誌詳細ページのURL}という辞書のリストで行うものとする。また、キーワードにマッチする図書・雑誌がなかった場合はNoneオブジェクトを返すように実装せよ。



In [118]:
def get_cinii_books(kw):
    endpoint = 'http://ci.nii.ac.jp/books/opensearch/search/'
    query = {
        'q': kw,
        'count': 10,
        'format': 'json'
    }
    response = requests.get(endpoint, params=query)
    result = list()
    if response.status_code == 200:
        for item in response.json()['@graph'][0]['items']:
            data = {
                'title': item['title'],
                'author': item.get('dc:creator'),
                'detail_url': item['@id']
            }
            result.append(data)
        return result
    else:
        return None

In [119]:
get_cinii_books('お好み焼き')

[{'title': 'お好み焼き食べたい : 具とソースで満腹バリエ33種',
  'author': None,
  'detail_url': 'https://ci.nii.ac.jp/ncid/BB0521742X'},
 {'title': '広島ベストスポット',
  'author': None,
  'detail_url': 'https://ci.nii.ac.jp/ncid/BA67030282'},
 {'title': 'お好み焼きの物語 : 執念の調査が解き明かす新戦前史',
  'author': '近代食文化研究会著',
  'detail_url': 'https://ci.nii.ac.jp/ncid/BB27669877'},
 {'title': '神戸とお好み焼き : まちづくりと比較都市論の視点から',
  'author': '三宅正弘著',
  'detail_url': 'https://ci.nii.ac.jp/ncid/BA59464673'},
 {'title': 'お好み焼きたこ焼きもんじゃ焼き&鉄板焼き',
  'author': None,
  'detail_url': 'https://ci.nii.ac.jp/ncid/BA51542558'},
 {'title': '「街的 (まちてき)」ということ : お好み焼き屋は街の学校だ',
  'author': '江弘毅著',
  'detail_url': 'https://ci.nii.ac.jp/ncid/BA78012806'},
 {'title': '浜内千波の野菜たっぷりやせる!お好み焼きダイエット',
  'author': '浜内千波著',
  'detail_url': 'https://ci.nii.ac.jp/ncid/BB04061314'},
 {'title': 'お好み焼き90%たこ焼き10%の本 : 粉もん100%レシピ&店ガイド',
  'author': None,
  'detail_url': 'https://ci.nii.ac.jp/ncid/BB02645347'},
 {'title': '絶品お好み焼き : たこ焼き・もんじゃ焼き : 人気店の味を家で楽しむ',
  'author': '成美堂